In [56]:
#import required libraries
import numpy as np
import pandas as pd
import math

In [57]:
np.random.seed(42)

# LOAD DATA

In [231]:
df_train = pd.read_csv('train_users_2.csv')
df_train.head()

id date_account_created  timestamp_first_active date_first_booking  \
0  gxn3p5htnn           2010-06-28          20090319043255                NaN   
1  820tgsjxq7           2011-05-25          20090523174809                NaN   
2  4ft3gnwmtx           2010-09-28          20090609231247         2010-08-02   
3  bjjt8pjhuk           2011-12-05          20091031060129         2012-09-08   
4  87mebub9p4           2010-09-14          20091208061105         2010-02-18   

      gender   age signup_method  signup_flow language affiliate_channel  \
0  -unknown-   NaN      facebook            0       en            direct   
1       MALE  38.0      facebook            0       en               seo   
2     FEMALE  56.0         basic            3       en            direct   
3     FEMALE  42.0      facebook            0       en            direct   
4  -unknown-  41.0         basic            0       en            direct   

  affiliate_provider first_affiliate_tracked signup_app first_device_type  \
0             direct               untracked        Web       Mac Desktop   
1             google               untracked        Web       Mac Desktop   
2             direct               untracked        Web   Windows Desktop   
3             direct               untracked        Web       Mac Desktop   
4             direct               untracked        Web       Mac Desktop   

  first_browser country_destination  
0        Chrome                 NDF  
1        Chrome                 NDF  
2            IE                  US  
3       Firefox               other  
4        Chrome                  US

In [232]:
df_age_gender = pd.read_csv('age_gender_bkts.csv')
df_age_gender.head()

age_bucket country_destination gender  population_in_thousands    year
0       100+                  AU   male                      1.0  2015.0
1      95-99                  AU   male                      9.0  2015.0
2      90-94                  AU   male                     47.0  2015.0
3      85-89                  AU   male                    118.0  2015.0
4      80-84                  AU   male                    199.0  2015.0

In [218]:
df_session = pd.read_csv('sessions.csv')
df_session.head()

user_id          action action_type        action_detail  \
0  d1mm9tcy42          lookup         NaN                  NaN   
1  d1mm9tcy42  search_results       click  view_search_results   
2  d1mm9tcy42          lookup         NaN                  NaN   
3  d1mm9tcy42  search_results       click  view_search_results   
4  d1mm9tcy42          lookup         NaN                  NaN   

       device_type  secs_elapsed  
0  Windows Desktop         319.0  
1  Windows Desktop       67753.0  
2  Windows Desktop         301.0  
3  Windows Desktop       22141.0  
4  Windows Desktop         435.0

In [219]:
df_countries = pd.read_csv('countries.csv')
df_countries.head()

country_destination  lat_destination  lng_destination  distance_km  \
0                  AU       -26.853388       133.275160   15297.7440   
1                  CA        62.393303       -96.818146    2828.1333   
2                  DE        51.165707        10.452764    7879.5680   
3                  ES        39.896027        -2.487694    7730.7240   
4                  FR        46.232193         2.209667    7682.9450   

   destination_km2 destination_language   language_levenshtein_distance  
0        7741220.0                   eng                           0.00  
1        9984670.0                   eng                           0.00  
2         357022.0                   deu                          72.61  
3         505370.0                   spa                          92.25  
4         643801.0                   fra                          92.06

# HANDLING NULL VALUES IN DATA

In [233]:
#Function to get the features containing null values
def get_null_features(df):
    na_count_df=df.isnull().sum()
    return na_count_df[na_count_df>0].index.values

In [234]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [235]:
for i in [df_train,df_age_gender,df_session,df_countries]:
    print("**************************************************************")
    if i.isnull().values.any():
        print("{} has null values in follwing features: {}".format(get_df_name(i),get_null_features(i)))
    else:
        print("{} does not have any null values".format(get_df_name(i)))
      

**************************************************************
df_train has null values in follwing features: ['date_first_booking' 'age' 'first_affiliate_tracked']
**************************************************************
df_age_gender does not have any null values
**************************************************************
df_session has null values in follwing features: ['user_id' 'action' 'action_type' 'action_detail' 'device_type'
 'secs_elapsed']
**************************************************************
df_countries does not have any null values


# CREATING BUCKETS FOR AGE IN df_train

In [236]:
age_buckets=df_age_gender['age_bucket'].unique()
age_buckets

array(['100+', '95-99', '90-94', '85-89', '80-84', '75-79', '70-74',
       '65-69', '60-64', '55-59', '50-54', '45-49', '40-44', '35-39',
       '30-34', '25-29', '20-24', '15-19', '10-14', '5-9', '0-4'],
      dtype=object)

In [224]:
np.unique(df_train['age'].dropna()).tolist()

[1.0,
 2.0,
 4.0,
 5.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 115.0,
 132.0,
 150.0,
 1924.0,
 1925.0,
 1926.0,
 1927.0,
 1928.0,
 1929.0,
 1931.0,
 1932.0,
 1933.0,
 1935.0,
 1936.0,
 1938.0,
 1942.0,
 1947.0,
 1949.0,
 1952.0,
 1953.0,
 1995.0,
 2008.0,
 2013.0,
 2014.0]

In [237]:
df=df_train

In [259]:
df_train=df

In [265]:
age_buckets[0]='100-2014'
for i in age_buckets:
    splits=i.split('-')
    #df_train['age_binned']=df_train['age'].apply(lambda x: 1 if (float(x) >= float(splits[0]) and float(x) <= float(splits[1]) and isinstance(x, float)) else x)
    

In [226]:
df_train['age_binned']=df_train['age_binned'].apply(lambda x: 'inappropriate age' if (x in ['100-2014','10-14','5-9','0-4']) else x)
    

In [266]:
df_train['age_binned']

0          NaN
1         38.0
2         56.0
3         42.0
4         41.0
5          NaN
6         46.0
7         47.0
8         50.0
9         46.0
10        36.0
11        47.0
12         NaN
13        37.0
14        36.0
15        33.0
16         NaN
17        31.0
18         NaN
19        29.0
20         NaN
21        30.0
22        40.0
23         NaN
24        40.0
25        26.0
26         NaN
27        32.0
28        35.0
29        37.0
          ... 
213421     NaN
213422     NaN
213423    20.0
213424    32.0
213425    30.0
213426     NaN
213427     NaN
213428     NaN
213429     NaN
213430    19.0
213431     NaN
213432    31.0
213433     NaN
213434     NaN
213435     NaN
213436     NaN
213437     NaN
213438     NaN
213439    43.0
213440    24.0
213441    34.0
213442     NaN
213443    36.0
213444     NaN
213445    23.0
213446    32.0
213447     NaN
213448    32.0
213449     NaN
213450     NaN
Name: age_binned, Length: 213451, dtype: float64